In [2]:
import zipfile

# Specify the path to the zip file
zip_file_path = "C:/Users/starf/OneDrive/Bureau/archive.zip"

# Specify the directory where you want to extract the contents
extract_to_path = "C:/Users/starf/OneDrive/Desktop/chess"

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

print("File extracted successfully!")

File extracted successfully!


In [4]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder



In [6]:

# 1. Load Dataset
data = pd.read_csv("C:/Users/starf/OneDrive/Bureau/archive/chessData.csv")  # Replace with your dataset file
print("Dataset Preview:\n", data.head())
data2 =pd.read_csv("C:/Users/starf/OneDrive/Bureau/archive/random_evals.csv")
data3 =pd.read_csv("C:/Users/starf/OneDrive/Bureau/archive/tactic_evals.csv")
print("Dataset Preview:\n", data2.head())
print("Dataset Preview:\n", data3.head())

Dataset Preview:
                                                  FEN Evaluation
0  rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...        -10
1  rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...        +56
2  rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...         -9
3  rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...        +52
4  rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...        -26
Dataset Preview:
                                                  FEN Evaluation
0  rnbqkb1r/pppppppp/B4n2/8/4P3/8/PPPP1PPP/RNBQK1...       -459
1  rnbqkb1r/pppppppp/5n2/1B6/4P3/8/PPPP1PPP/RNBQK...       -125
2  rnbqkbnr/p1pppppp/8/1p6/4P3/8/PPPP1PPP/RNBQKBN...       +198
3  rnbqkb1r/pppppppp/5n2/8/4P3/7N/PPPP1PPP/RNBQKB...       -155
4  rnbqkbnr/ppppp1pp/8/5p2/4P3/8/PPPP1PPP/RNBQKBN...       +209
Dataset Preview:
                                                  FEN Evaluation  Move
0  r2qkbr1/pb1nn3/1ppp3p/8/3P1p2/2PB1N1P/PPQN1PP1...        #+2  d3g6
1  r2qkb2/pb1nn3/1ppp2rp/8/3P1p2/2P2N1

In [8]:
data
data2
data3

,FEN,Evaluation,Move
0,r2qkbr1/pb1nn3/1ppp3p/8/3P1p2/2PB1N1P/PPQN1PP1...,#+2,d3g6
1,r2qkb2/pb1nn3/1ppp2rp/8/3P1p2/2P2N1P/PPQN1PP1/...,#+1,c2g6
2,r2qkbr1/pb1nn3/1ppp2Bp/8/3P1p2/2P2N1P/PPQN1PP1...,#+1,g8g6
3,8/7p/R5p1/2p1pkP1/7P/P4PK1/1r6/3q4 w - - 6 46,#+1,a6f6
4,6k1/pp6/3p4/2p1p3/2P1P1q1/1P1P2pP/P5P1/5K2 w -...,+408,h3g4
...,...,...,...
2628214,3r4/3pk3/6q1/Qp1pPpP1/1PbPn3/4B3/5RKP/8 w - - ...,-1053,a5a3
2628215,r2qkbnr/5ppp/p1p1p3/3p4/3P2PP/2N5/PPb1QP2/R1B1...,+215,h7h5
2628216,Q5b1/6pk/3q1pNp/1p1p3N/3P1P2/2P5/1P3KPP/8 b - ...,+944,h7g6
2628217,3rb3/ppk1r2p/6p1/2P5/4B3/P7/6PP/3Q3K w - - 0 35,-692,d1c2


In [10]:
# 2. Preprocess Data
# Assuming the dataset has columns: 'FEN' (board state) and 'Move' (next move)
fen_strings = data['FEN'].values
moves = data3['Move'].values


In [12]:
# Encode the moves into integers
move_encoder = LabelEncoder()
encoded_moves = move_encoder.fit_transform(moves)

In [14]:
print(f"Number of FEN strings: {len(fen_strings)}")
print(f"Number of encoded moves: {len(encoded_moves)}")


Number of FEN strings: 12958035
Number of encoded moves: 2628219


In [16]:
min_length = min(len(fen_strings), len(encoded_moves))


In [18]:
min_length

2628219

In [20]:
fen_strings = fen_strings[:min_length]
encoded_moves = encoded_moves[:min_length]


In [22]:
print(f"Updated number of FEN strings: {len(fen_strings)}")
print(f"Updated number of encoded moves: {len(encoded_moves)}")


Updated number of FEN strings: 2628219
Updated number of encoded moves: 2628219


In [24]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(fen_strings, encoded_moves, test_size=0.2, random_state=42)

In [28]:
import numpy as np

# Define a fixed size for the feature vectors
VECTOR_SIZE = 100

def fen_to_vector(fen):
    # Convert the FEN to a simple ASCII representation of fixed size (VECTOR_SIZE)
    vector = np.array([ord(c) for c in fen if c.isalnum()])  # ASCII encoding of alphanumeric chars
    # Pad or truncate the vector to the fixed size
    if len(vector) < VECTOR_SIZE:
        # Pad with zeros if the vector is shorter than the desired size
        vector = np.pad(vector, (0, VECTOR_SIZE - len(vector)), mode='constant', constant_values=0)
    else:
        # Truncate the vector if it's longer than the desired size
        vector = vector[:VECTOR_SIZE]
    return vector



In [30]:
# Convert the FEN strings to feature vectors
X_train = np.array([fen_to_vector(fen) for fen in X_train])
X_test = np.array([fen_to_vector(fen) for fen in X_test])

print(X_train.shape)  # Check the shape of the feature vectors


(2102575, 100)


In [32]:
# Pad sequences to the same length
max_length = max(max(len(x) for x in X_train), max(len(x) for x in X_test))
X_train = np.array([np.pad(x, (0, max_length - len(x)), 'constant') for x in X_train])
X_test = np.array([np.pad(x, (0, max_length - len(x)), 'constant') for x in X_test])

In [38]:
from tensorflow.keras.utils import to_categorical

In [40]:
# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=len(move_encoder.classes_))
y_test = to_categorical(y_test, num_classes=len(move_encoder.classes_))

In [42]:
# Build the LSTM Model
model = Sequential([
    Embedding(input_dim=256, output_dim=128, input_length=max_length),
    LSTM(128, return_sequences=False),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(len(move_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\starf\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Train the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32)

MemoryError: Unable to allocate 15.2 GiB for an array with shape (2102575, 1943) and data type float32

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


In [ ]:
# Predict a move for a sample FEN string
sample_fen = X_test[0]
predicted_class = np.argmax(model.predict(np.expand_dims(sample_fen, axis=0)))
predicted_move = move_encoder.inverse_transform([predicted_class])
print("Predicted Move:", predicted_move)

In [ ]:
# Evaluate with classification report
y_test_decoded = np.argmax(y_test, axis=1)
y_pred_decoded = np.argmax(model.predict(X_test), axis=1)
print("Classification Report:\n", classification_report(y_test_decoded, y_pred_decoded))